## Data Ingestion stage

In [164]:
from collections import namedtuple
import os

In [165]:
%cd Dog-Cat-Classifier/

[WinError 2] The system cannot find the file specified: 'Dog-Cat-Classifier/'
c:\Projects\Dog-Cat-Classifier


In [166]:
%pwd

'c:\\Projects\\Dog-Cat-Classifier'

In [167]:
DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [168]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [169]:
from src.Dog_Cat_Classifier.constants import *
from src.Dog_Cat_Classifier.utils import read_yaml, create_directory

In [170]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = os.path.normpath(str(CONFIG_FILE_PATH)),
        params_filepath = os.path.normpath(str(PARAMS_FILE_PATH))):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directory([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [171]:
import os
import urllib.request as request
import requests
from zipfile import ZipFile, is_zipfile

In [172]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        try:
            # Check if the file already exists
            if not os.path.exists(self.config.local_data_file):
                print(f"Downloading file from {self.config.source_URL}...")

                # Check content type before downloading
                response = requests.head(self.config.source_URL)
                content_type = response.headers.get('Content-Type', '')

                # Stream the download to handle large files
                with requests.get(self.config.source_URL, stream=True) as response:
                    response.raise_for_status()  # Raise an error for a bad status code
                    
                    # Open the destination file in binary write mode
                    with open(self.config.local_data_file, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=8192):  # Read in 8KB chunks
                            f.write(chunk)

                print(f"File downloaded successfully: {self.config.local_data_file}")
            else:
                print("File already exists. Skipping download.")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading file: {e}")
            raise

    def _get_updated_list_of_files(self, list_of_files):
        # Filter the list of files to include only .jpg files that contain 'Cat' or 'Dog' in their names
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]
    
    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        # Define the target file path
        target_filepath = os.path.join(working_dir, f)
        
        # Extract the file if it does not already exist
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        # Remove the file if it is empty
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

    def unzip_and_clean(self):
        # Check if the local data file is a valid ZIP file
        if not is_zipfile(self.config.local_data_file):
            raise ValueError(f"The file {self.config.local_data_file} is not a valid ZIP file.")
        
        # Open the ZIP file and process its contents
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

In [173]:
try:
    # Initialize the ConfigurationManager
    config = ConfigurationManager()
    
    # Get the data ingestion configuration
    data_ingestion_config = config.get_data_ingestion_config()
    
    # Initialize the DataIngestion class with the configuration
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    # Download the data file
    data_ingestion.download_file()
    
    # Unzip and clean the downloaded data file
    data_ingestion.unzip_and_clean()
except Exception as e:
    # Raise any exceptions that occur
    raise e

[2024-12-31 18:23:51,059 - INFO - Read the yaml file: config\config.yaml]
[2024-12-31 18:23:51,060 - INFO - Read the yaml file: params.yaml]
[2024-12-31 18:23:51,061 - INFO - Created directory: artifacts]
[2024-12-31 18:23:51,062 - INFO - Created directory: artifacts/data_ingestion]
File downloaded successfully: artifacts/data_ingestion/data.zip
